Segmenting and Clustering neighboehood in Toronto
Part 1: Prepare dataframe fetching data from webpage table

In [1]:
# import libraries
import requests
import lxml.html as lh
import pandas as pd

In [2]:
# setting url for the source data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Create a handle, page, to handle the contents of the website
page = requests.get(url)
# Store the contents of the website under doc
doc = lh.fromstring(page.content)
# Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

For sanity check, ensure that all the rows have the same width. If not, we probably got something more than just the table.

In [3]:
# Check the length of the first 12 rows (Sanity Check)
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
# Create empty list
col=[]
i=0
# For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [5]:
# Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T  is our j'th row
    T=tr_elements[j]
    
    # If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    # i is the index of our column
    i=0
    
    # Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        # Check if row is empty
        if i>0:
        # Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        # Append the data to the empty list of the i'th column
        col[i][1].append(data)
        # Increment i for the next column
        i+=1

Just to be sure, let’s check the length of each column. Ideally, they should all be the same.

In [6]:
[len(C) for (title,C) in col]

[181, 181, 181]

Create the DataFrame

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

Looking at the top 5 cells on the DataFrame

In [8]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Renaming column headings to remove \n and to phrase them as needed

In [9]:
df.rename(columns={"Postal Code\n": "Postal Code", "Borough\n": "Borough", "Neighborhood\n": "Neighborhood"}, inplace=True)


In [10]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Ignore cells with a borough that is 'Not assigned'

In [11]:
df = df[~df['Borough'].str.contains('Not assigned')]

In [12]:
df

,Postal Code,Borough,Neighborhood
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
...,...,...,...
165,M4Y\n,Downtown Toronto\n,Church and Wellesley\n
168,M7Y\n,East Toronto\n,"Business reply mail Processing Centre, South C..."
169,M8Y\n,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


Remove last undesired row

In [13]:
df = df[:-1]

In [14]:
df.tail()

,Postal Code,Borough,Neighborhood
160,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n"
165,M4Y\n,Downtown Toronto\n,Church and Wellesley\n
168,M7Y\n,East Toronto\n,"Business reply mail Processing Centre, South C..."
169,M8Y\n,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


Remove \n from all rows

In [15]:
df_final = df.replace("\n", "", regex=True)

In [16]:
df_final.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Restting the index to modified dataframe

In [17]:
df_final.reset_index(inplace=True, drop=True)

In [18]:
df_final.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Checking shape and value counts to ensure no duplicates of Postal Codes

In [19]:
df_final['Postal Code'].value_counts()

M4E    1
M6N    1
M6C    1
M6E    1
M3J    1
      ..
M6J    1
M2L    1
M6H    1
M4M    1
M1B    1
Name: Postal Code, Length: 103, dtype: int64

In [20]:
df_final.shape

(103, 3)

Since Value Counts are equal to number of rows (103), there are no duplicate Postal Codes, for which 'Neighborhood' needs to be combined.

----*----*---- End of Part-1

Part -2 : Create Dataframe of Geospacial Data

In [21]:
import pandas as pd

In [22]:
# Create a dataframe from csv file
df_gsc = pd.read_csv("Geospatial_Coordinates.csv")

In [23]:
# display top 5 rows
df_gsc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# dimensions of the dataframe
df_gsc.shape

(103, 3)

In [25]:
# set common index on both the dataframes
df_final.set_index('Postal Code')

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


In [26]:
# set common index on both the dataframes
df_gsc.set_index('Postal Code')

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [27]:
# merge both the dataframes in a third dataframe
df_merged = df_final.merge(df_gsc, on="Postal Code", left_index=True)

In [28]:
# display top 12 rows
df_merged.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
25,M3A,North York,Parkwoods,43.753259,-79.329656
34,M4A,North York,Victoria Village,43.725882,-79.315572
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
71,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
85,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
93,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
26,M3B,North York,Don Mills,43.745906,-79.352188
35,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


----*----*---- End of Part-2